# Linear Algebra

Linear Algebra is a discipline where vector spaces and linear mapping between them are studied. In physics and astronomy, several phenomena can be readily written in terms of linear variables, what makes Computational Linear Algebra a very important topic to be covered throughout this course. We shall cover linear systems of equations, techniques for calculating inverses and determinants and factorization methods.

An interesting fact of Computational Linear Algebra is that it does not comprises numerical approaches as most of the methods are exact. The usage of a computer is then necessary because of the large number of calculations rather than the non-soluble nature of the problems. Numerical errors come then from round-off approximations.

- - -
- [Linear Systems of Equations](#Linear-Systems-of-Equations) 
    - [Matrices and vectors](#Matrices-and-vectors)
    - [Example 1](#Example-1)
    - [Matrices in Python](#Matrices-in-Python)
    - [Basic operations with matrices](#Basic-operations-with-matrices)
- [Gaussian Elimination](#Gaussian-Elimination)
    - [General Gaussian elimination](#General-Gaussian-elimination)
    - [Computing time](#Computing-time)
    - [Example 2](#Example-2)
- [Pivoting Strategies](#Pivoting-Strategies)
    - [Partial pivoting](#Partial-pivoting)
- [Matrix Inversion](#Matrix-Inversion)
- [Determinant of a Matrix](#Determinant-of-a-Matrix)
    - [Calculating determinants](#Calculating-determinants)
    - [Computing time of determinants](#Computing-time-of-determinants)
    - [Properties of determinants](#Properties-of-determinants)
- [LU Factorization](#LU-Factorization)
    - [Derivation of LU factorization](#Derivation-of-LU-factorization)
    - [Algorithm for LU factorization](#Algorithm-for-LU-factorization)

- - -

In [45]:
from IPython.display import Math, Latex, HTML, Markdown, YouTubeVideo
%pylab inline

import sympy
x =sympy.Symbol('x') # declare analytical varibles
sympy.init_printing() # Use LaTeX to print sympy obejects

import numpy as np
import matplotlib.pyplot as plt
# JSAnimation import available at https://github.com/jakevdp/JSAnimation
from JSAnimation import IPython_display
from matplotlib import animation
#Interpolation add-on
import scipy.interpolate as interp
xrange=range

def sprint(x):
    return sympy.Matrix(x)

spe_print=vectorize(sprint)

Populating the interactive namespace from numpy and matplotlib


- - - 

# Linear Systems of Equations

## Matrix diagonalization
We start again with the matrix equation, capitol bold letters denotes matrices
\begin{equation}
  \boldsymbol{A}\boldsymbol{X}=\boldsymbol{B}\,,
\end{equation}
where $\boldsymbol{A}$ is an $n \times n$ matrix.

We know that there exists a bi-diagonal transformación such that
\begin{equation}
  \boldsymbol{V}^\dagger\boldsymbol{A}\,\boldsymbol{U}=\boldsymbol{A}_{\text{diag}}
\end{equation}
So, by doing standard operations we have
\begin{align}
  \boldsymbol{V}^\dagger \boldsymbol{A} \boldsymbol{U} \boldsymbol{U}^\dagger \boldsymbol{X}=& \boldsymbol{V}^\dagger \boldsymbol{B}\\
  \boldsymbol{A}_{\text{diag}} \boldsymbol{X}'=&\boldsymbol{B}'\,,      
\end{align}
where
\begin{align}
  \boldsymbol{X}=& \boldsymbol{U}\boldsymbol{X}'\,, &    \boldsymbol{B}=& \boldsymbol{V}\boldsymbol{B}'\,.
\end{align}

If $\boldsymbol{A}_{\text{diag}}=\operatorname{diag}(\lambda_1,\lambda_2,\ldots \lambda_n)$, $\boldsymbol{X}^{\operatorname{T}}=(x_1\ x_2\ x_3)^{\operatorname{T}}$ and $\boldsymbol{B}^{\operatorname{T}}=(b_1\ b_2\ b_3)^{\operatorname{T}}$,
the solution of the system is given by
\begin{equation}
\lambda_i x'_i=b_i'\,.
\end{equation}

 A suitable way to introduce this method is applying it to some basic problem. To do so, let's take the result of the [Example 1](#Example-1):

$$ \begin{bmatrix}
5 & -4 & 0 \\
-4 & 7 & -3 \\ 
0 & -3 & 5
\end{bmatrix}
\begin{bmatrix}
x_{1} \\
x_{2} \\
x_{3} 
\end{bmatrix}  =
\begin{bmatrix}
1 \\
0 \\
-2
\end{bmatrix}
$$
As the matrix is symmetric $\boldsymbol{V}=\boldsymbol{U}$ and $\boldsymbol{U}^\dagger=\boldsymbol{U}^{\operatorname{T}}$ 

In [25]:
M1=np.array([[5,-4,0],[-4,7,-3],[0,-3,5]])

In [26]:
A=M1

In [27]:
B=np.array([[1],[0],[-2]])
sprint(B)

⎡1 ⎤
⎢  ⎥
⎢0 ⎥
⎢  ⎥
⎣-2⎦

In [28]:
A_diag,V=np.linalg.eig( A )

In [36]:
sprint(A_diag.round(2))

⎡11.1⎤
⎢    ⎥
⎢0.9 ⎥
⎢    ⎥
⎣5.0 ⎦

In [30]:
sprint(V.round(14))

⎡-0.50719112448443  -0.61867371307032  -0.6⎤
⎢                                          ⎥
⎢ 0.7733421413379   -0.63398890560554  0.0 ⎥
⎢                                          ⎥
⎣-0.38039334336332  -0.46400528480274  0.8 ⎦

We first check the proper order of the diagonalization

In [42]:
sprint(np.dot(V.transpose(),V).round(14))

⎡1.0  0.0  0.0⎤
⎢             ⎥
⎢0.0  1.0  0.0⎥
⎢             ⎥
⎣0.0  0.0  1.0⎦

In [39]:
sprint(np.dot(  np.dot( V.transpose(),A  ), V).round(14))

⎡11.0990195135928        0.0         0.0⎤
⎢                                       ⎥
⎢      0.0         0.90098048640722  0.0⎥
⎢                                       ⎥
⎣      0.0               0.0         5.0⎦

If we make
$$ \boldsymbol{U}=[\boldsymbol{U}_1,\boldsymbol{U}_2,\boldsymbol{U}_3], \qquad \boldsymbol{V}=[\boldsymbol{V}_1,\boldsymbol{V}_2,\boldsymbol{V}_3] $$

We know that there exists a bi-diagonal transformación such that
\begin{equation}
  \boldsymbol{A}\,\boldsymbol{U}=\boldsymbol{V}\boldsymbol{A}_{\text{diag}}
\end{equation}
\begin{equation}
  \boldsymbol{A}\,\boldsymbol{U}_i=\lambda_i\boldsymbol{V}_i
\end{equation}

We can use this to check the proper order of the eigenvalues

Since

In [53]:
V0=np.reshape( V[:,0],(3,1))
V1=np.reshape( V[:,1],(3,1))
V2=np.reshape( V[:,2],(3,1))
#display(V1)
#display(V2)
#V3
sprint(V0),sprint(V1),sprint(V2)

⎛⎡-0.50719112448443 ⎤  ⎡-0.618673713070322⎤  ⎡        -0.6        ⎤⎞
⎜⎢                  ⎥  ⎢                  ⎥  ⎢                    ⎥⎟
⎜⎢0.773342141337902 ⎥, ⎢-0.633988905605538⎥, ⎢1.91548673921395e-16⎥⎟
⎜⎢                  ⎥  ⎢                  ⎥  ⎢                    ⎥⎟
⎝⎣-0.380393343363323⎦  ⎣-0.464005284802741⎦  ⎣        0.8         ⎦⎠

In [56]:
sprint(np.dot(A,V0))

⎡-5.62932418777376⎤
⎢                 ⎥
⎢8.58333951739301 ⎥
⎢                 ⎥
⎣-4.22199314083032⎦

In [57]:
sprint(A_diag[0]*V0)

⎡-5.62932418777376⎤
⎢                 ⎥
⎢8.58333951739301 ⎥
⎢                 ⎥
⎣-4.22199314083032⎦

In [58]:
sprint(np.dot(A,V1))

⎡-0.557412942929459⎤
⎢                  ⎥
⎢-0.571211632549253⎥
⎢                  ⎥
⎣-0.418059707197094⎦

In [59]:
sprint(A_diag[1]*V1)

⎡-0.557412942929456⎤
⎢                  ⎥
⎢-0.571211632549255⎥
⎢                  ⎥
⎣-0.418059707197092⎦

In [60]:
sprint(np.dot(A,V2))

⎡        -3.0        ⎤
⎢                    ⎥
⎢3.11719755685002e-15⎥
⎢                    ⎥
⎣        4.0         ⎦

In [61]:
sprint(A_diag[2]*V2)

⎡        -3.0        ⎤
⎢                    ⎥
⎢9.57743369606976e-16⎥
⎢                    ⎥
⎣        4.0         ⎦

In [81]:
U=hstack((V0,V1,V2))
sprint(np.dot(np.dot(U.transpose(),A),U).round(14))

⎡11.0990195135928        0.0         0.0⎤
⎢                                       ⎥
⎢      0.0         0.90098048640722  0.0⎥
⎢                                       ⎥
⎣      0.0               0.0         5.0⎦

In [84]:
Uorder=hstack((V0,V2,V1))

In [116]:
sprint(np.dot(np.dot(Uorder.transpose(),A),Uorder).round(14))

⎡11.0990195135928  0.0        0.0       ⎤
⎢                                       ⎥
⎢      0.0         5.0        0.0       ⎥
⎢                                       ⎥
⎣      0.0         0.0  0.90098048640722⎦

In [140]:
Bp=np.dot(np.linalg.inv(V),B)
Xp=np.dot(np.linalg.inv(np.diag(A_diag)),Bp).round(14)

In [141]:
sprint(Xp)

⎡0.02284846530197⎤
⎢                ⎥
⎢0.34333358069572⎥
⎢                ⎥
⎣     -0.44      ⎦

In [150]:
X=np.dot(V,Xp).round(2)
sprint(X)

⎡0.04 ⎤
⎢     ⎥
⎢-0.2 ⎥
⎢     ⎥
⎣-0.52⎦

In [147]:
sprint(np.dot(A,X).round(14))

⎡1.0 ⎤
⎢    ⎥
⎢0.0 ⎥
⎢    ⎥
⎣-2.0⎦

In [148]:
sprint(B)

⎡1 ⎤
⎢  ⎥
⎢0 ⎥
⎢  ⎥
⎣-2⎦